In [1]:
import ee
import geemap
import pandas as pd
from ipyleaflet import ImageOverlay
from ipyleaflet import WidgetControl
from ipywidgets import VBox, HTML


# Authentifier et initialiser Google Earth Engine
ee.Authenticate()
ee.Initialize()

In [2]:
# Section 1 : Données Vectorielles
# Créer une carte interactive pour afficher les données vectorielles et raster
Map = geemap.Map()

# Importer les données de shapefiles pour les différents niveaux administratifs du Niger
pays = ee.FeatureCollection("projects/ee-senemalick123450/assets/geoBoundaries-NER-ADM0")  # Niveau pays
regions = ee.FeatureCollection("projects/ee-senemalick123450/assets/geoBoundaries-NER-ADM1")  # Niveau régions
departements = ee.FeatureCollection("projects/ee-senemalick123450/assets/geoBoundaries-NER-ADM2")  # Niveau départements
communes = ee.FeatureCollection("projects/ee-senemalick123450/assets/geoBoundaries-NER-ADM3")  # Niveau communes

# Calculs statistiques : récupérer et afficher des informations pour le niveau pays
country_count = pays.size().getInfo()  # Obtenir le nombre de géométries dans le shapefile des pays
print(f'Nombre de géométries (pays) : {country_count}')

crs_country = pays.first().geometry().projection().getInfo()  # Obtenir le système de coordonnées (CRS) du shapefile
print(f'CRS du shapefile du pays : {crs_country}')

extent_country = pays.geometry().bounds().getInfo()  # Obtenir les limites géographiques du shapefile
print(f'Extent (limites) du pays : {extent_country}')

# Visualisation des shapefiles : ajouter le shapefile du niveau pays sur la carte
Map.addLayer(pays, {}, 'Pays')  # Ajouter le layer du pays sans style spécifique
Map.centerObject(pays, 6)  # Centrer la vue sur le shapefile du pays avec un niveau de zoom de 6

# Section 2 : Données Raster - Visualisation de la population
# Charger un raster de population (ex. WorldPop ou une source de données de population pour le Niger)
population_raster = ee.Image("projects/ee-senemalick123450/assets/NER_population_v1_0_gridded")  # Remplacez par votre raster si nécessaire

# Ajouter le raster de population à la carte avec une palette de couleurs pour représenter les densités de population
palette = ['ffffcc', 'c2e699', '78c679', '31a354', '006837']  # Palette de couleurs allant du clair au foncé pour illustrer les variations de densité
Map.addLayer(population_raster, {'palette': palette, 'min': 0, 'max': 300}, 'Population')  # Définir la palette et les valeurs min/max de population

# Vérifier la résolution spatiale du raster en obtenant la taille des pixels
pixel_size = population_raster.projection().nominalScale().getInfo()  # Obtenir la résolution (taille des pixels en mètres)
print(f"Taille des pixels du raster de population : {pixel_size} mètres")

# Afficher la carte combinée (données vectorielles + données raster)
Map  # Affiche la carte interactive avec les couches de données ajoutées


Nombre de géométries (pays) : 1
CRS du shapefile du pays : {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
Extent (limites) du pays : {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[0.16666781762811478, 11.691999274801383], [15.996668338772286, 11.691999274801383], [15.996668338772286, 23.517178579212032], [0.16666781762811478, 23.517178579212032], [0.16666781762811478, 11.691999274801383]]]}
Taille des pixels du raster de population : 92.76624195800512 mètres


Map(center=[17.413175629395553, 9.346513612635333], controls=(WidgetControl(options=['position', 'transparent_…

In [3]:
import pandas as pd

# Fonction pour calculer la population dans une zone administrative donnée
def calculer_population_par_zone(admin_level, zone_name):
    pop_totale = population_raster.reduceRegion(
        reducer=ee.Reducer.sum(),  # Utilisation du réducteur de somme pour agréger la population dans la zone
        geometry=admin_level.geometry(),  # Définition de la géométrie de la zone administrative
        scale=population_raster.projection().nominalScale(),  # Échelle basée sur la résolution du raster de population
        maxPixels=1e13  # Limite de pixels pour la réduction
    ).get('b1')  # 'b1' correspond à la bande de population par défaut dans WorldPop

    return ee.Feature(None, {'ID': zone_name, 'Population': pop_totale})  # Création d'une Feature avec l'ID de la zone et la population calculée

# Calculer la population pour chaque niveau administratif en appliquant la fonction sur chaque Feature
pop_pays = pays.map(lambda feature: calculer_population_par_zone(feature, feature.get('shapeID')))  # Population par pays
pop_regions = regions.map(lambda feature: calculer_population_par_zone(feature, feature.get('shapeID')))  # Population par région
pop_departements = departements.map(lambda feature: calculer_population_par_zone(feature, feature.get('shapeID')))  # Population par département
pop_communes = communes.map(lambda feature: calculer_population_par_zone(feature, feature.get('shapeID')))  # Population par commune

# Fonction pour convertir une FeatureCollection en dictionnaire avec les ID et les populations
def features_to_dict(feature_collection):
    ids = feature_collection.aggregate_array('ID').getInfo()  # Extraction des IDs de chaque Feature
    populations = feature_collection.aggregate_array('Population').getInfo()  # Extraction des valeurs de population
    return [{'ID': id, 'Population': pop} for id, pop in zip(ids, populations)]  # Construction d'une liste de dictionnaires avec les données

# Obtenir les données sous forme de liste de dictionnaires pour chaque niveau administratif
pop_pays_data = features_to_dict(pop_pays)  # Données pour les pays
pop_regions_data = features_to_dict(pop_regions)  # Données pour les régions
pop_departements_data = features_to_dict(pop_departements)  # Données pour les départements
pop_communes_data = features_to_dict(pop_communes)  # Données pour les communes

# Convertir les résultats en DataFrames pandas et les exporter en fichiers CSV
pop_pays_df = pd.DataFrame(pop_pays_data)  # DataFrame pour les pays
pop_regions_df = pd.DataFrame(pop_regions_data)  # DataFrame pour les régions
pop_departements_df = pd.DataFrame(pop_departements_data)  # DataFrame pour les départements
pop_communes_df = pd.DataFrame(pop_communes_data)  # DataFrame pour les communes

# Exportation de chaque DataFrame au format CSV
pop_pays_df.to_csv('population_par_pays.csv', index=False)  # Export des données de population par pays
pop_regions_df.to_csv('population_par_regions.csv', index=False)  # Export des données de population par régions
pop_departements_df.to_csv('population_par_departements.csv', index=False)  # Export des données de population par départements
pop_communes_df.to_csv('population_par_communes.csv', index=False)  # Export des données de population par communes

# Message de confirmation
print("Les fichiers CSV de population par niveau administratif ont été exportés avec succès.")


Les fichiers CSV de population par niveau administratif ont été exportés avec succès.


In [4]:
# Définir la nouvelle résolution pour l'agrégation en mètres (ici, 5 km = 5000 mètres)
nouvelle_resolution = 5000

# Agréger les pixels du raster de population à la nouvelle résolution définie
population_raster_5km = population_raster.reduceResolution(
    reducer=ee.Reducer.sum(),  # Utiliser la somme pour agréger les valeurs de population dans chaque zone  # Permet à Google Earth Engine d'ajuster automatiquement l'agrégation si nécessaire
    maxPixels=5000  # Limite du nombre de pixels à traiter dans chaque agrégation pour éviter une surcharge de calcul
).reproject(
    crs=population_raster.projection(),  # Conserver le système de coordonnées d'origine du raster
    scale=nouvelle_resolution  # Appliquer la nouvelle résolution de 5 km
)

# Ajouter le raster agrégé avec la nouvelle résolution sur la carte pour visualisation
Map.addLayer(
    population_raster_5km, 
    {'palette': palette, 'min': 0, 'max': 3000},  # Appliquer la même palette de couleurs pour une échelle de population
    'Population (5km resolution)'  # Légende pour indiquer la résolution du raster
)

# Afficher la carte avec le raster de population agrégé
Map


Map(center=[17.413175629395553, 9.346513612635333], controls=(WidgetControl(options=['position', 'transparent_…

In [6]:
# Calculer le raster représentant le pourcentage d'enfants âgés de 2 à 10 ans dans la population.
# Hypothèse : les enfants de 2 à 10 ans représentent 0,1 % de la population totale
population_enfants_2_10 = population_raster_5km.multiply(0.001)

# Ajouter le raster calculé pour la population d'enfants (2-10 ans) à la carte pour visualisation
# La palette de couleurs et la plage (min et max) sont définies pour représenter les valeurs du pourcentage d'enfants
Map.addLayer(
    population_enfants_2_10, 
    {'palette': palette, 'min': 0, 'max': 3},  # Palette de couleurs et limites de l'échelle pour la population d'enfants
    'Population Enfants (2-10 ans, 5km resolution)'  # Légende pour indiquer la tranche d'âge et la résolution
)

# Afficher la carte avec le raster des enfants âgés de 2-10 ans
Map


Map(bottom=7687.0, center=[17.413175629395553, 9.346513612635333], controls=(WidgetControl(options=['position'…

In [7]:
# Fonction pour obtenir les images raster par année et sélectionner la première bande
def importation_rasters(annee):
    nom_raster = f"202406_Global_Pf_Parasite_Rate_NER_{annee}"
    asset_id = f'projects/ee-senemalick123450/assets/{nom_raster}'
    return ee.Image(asset_id).select(0)

# Liste des années pour les rasters
annees = list(range(2000, 2023))

# Charger toutes les images raster et sélectionner la première bande
rasters = [importation_rasters(annee) for annee in annees]

# Créer une collection à partir de la liste d'images
stack_raster = ee.ImageCollection(rasters)

In [8]:
# Calculer la moyenne sur la collection
moyenne_raster = stack_raster.mean()

In [9]:
# Paramètres de visualisation pour l'image raster
# 'min' et 'max' définissent les valeurs minimales et maximales de la plage de couleurs
# 'palette' définit les couleurs allant du plus bas au plus haut
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Création d'une instance de carte centrée sur les coordonnées [0, 0] avec un zoom initial de 2
Map = geemap.Map(center=[0, 0], zoom=2)

# Fonction pour visualiser une image sur la carte en ajoutant une couche avec des paramètres spécifiques
# 'image' est l'image raster à afficher, 'vis_params' sont les paramètres de visualisation, et 'title' est le nom de la couche
def visualize_image(image, vis_params, title):
    Map.addLayer(image, vis_params, title)

# Appel de la fonction pour visualiser 'moyenne_raster' avec les paramètres définis
visualize_image(moyenne_raster, vis_params, 'moyenne_raster')

# Ajout d'une barre de couleurs (colorbar) pour représenter les valeurs de l'image
# 'label' définit le texte affiché sur la légende, et 'layer_name' le nom de la couche associée
Map.add_colorbar(vis_params=vis_params, label="Valeur", layer_name='moyenne_raster')

# Ajouter une flèche du nord sur la carte pour indiquer l'orientation
# 'north_arrow_url' est le lien de l'image, qui sera affichée comme une flèche du nord
north_arrow_url = 'https://img.freepik.com/vecteurs-premium/icone-fleche-nord-symbole-compas-panneau-direction-carte-logo-nord-sud-ouest-est-dans-vecteur-plat_754658-414.jpg?w=2000'

# Définition d'une superposition d'image avec les paramètres de la flèche du nord
# 'bounds' définit la position et la taille de la flèche sur la carte
north_arrow = ImageOverlay(
    url=north_arrow_url,
    bounds=[[20, -10], [25, -15]],  # Coordonnées pour réduire la taille et positionner la flèche en haut à droite
    opacity=0.8  # Opacité de l'image, 0 étant transparent et 1 opaque
)
Map.add_layer(north_arrow)  # Ajout de la flèche du nord à la carte

# Affichage final de la carte avec toutes les couches ajoutées
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [10]:
# Paramètres de visualisation du raster
vis_params = {
    'min': 0,
    'max': 1,
    #'palette': ['blue', 'green', 'yellow', 'red']
}
# Créer une carte
Map = geemap.Map(center=[0, 0], zoom=2)

# Ajouter le raster à la carte
Map.addLayer(moyenne_raster, vis_params, 'Raster')

# Ajouter le shapefile à la carte
Map.addLayer(pays, {'color': 'black'}, 'Shapefile')

# Afficher la carte avec les deux couches superposées
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [12]:
# Fonction pour importer les images raster par année
# ------------------------
# Cette fonction prend une année en paramètre, construit le nom du raster et renvoie l'image raster correspondante
def importation_rasters(annee):
    # Créer le nom du raster en fonction de l'année donnée
    nom_raster = f"202406_Global_Pf_Parasite_Rate_NER_{annee}"
    # Construire l'ID de l'asset dans Google Earth Engine
    asset_id = f'projects/ee-senemalick123450/assets/{nom_raster}'
    # Retourner l'image raster en sélectionnant la première bande (index 0)
    return ee.Image(asset_id).select(0)

# ------------------------
# Charger les rasters et créer une collection
# ------------------------
# Liste des années pour les rasters de 2000 à 2022
annees = list(range(2000, 2023))

# Charger toutes les images raster en utilisant la fonction définie précédemment
# Cette liste contient toutes les images raster importées pour chaque année
rasters = [importation_rasters(annee) for annee in annees]

# Créer une collection d'images à partir de la liste d'images raster
stack_raster = ee.ImageCollection(rasters)

# Calculer la moyenne et l'écart type des valeurs de pixels pour toutes les images de la collection
moyenne_raster = stack_raster.mean()
ecart_type_raster = stack_raster.reduce(ee.Reducer.stdDev())

# Paramètres de visualisation pour la moyenne et l'écart type
vis_params_moyenne = {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'green', 'yellow', 'red']  # Palette de couleurs pour visualiser la moyenne
}

vis_params_ecart_type = {
    'min': 0,
    'max': 0.5,
    'palette': ['white', 'orange', 'red']  # Palette de couleurs pour visualiser l'écart type
}

# Créer une carte pour visualiser la moyenne et l'écart type
Map_stats = geemap.Map(center=[12, -1], zoom=6)  # Initialiser la carte centrée sur des coordonnées spécifiques
Map_stats.addLayer(moyenne_raster, vis_params_moyenne, 'Moyenne Taux Malaria (2000-2022)')  # Ajouter la couche de moyenne
Map_stats.addLayer(ecart_type_raster, vis_params_ecart_type, 'Ecart Type Taux Malaria (2000-2022)')  # Ajouter la couche d'écart type

# Ajouter des barres de couleur pour les deux couches
Map_stats.add_colorbar(vis_params=vis_params_moyenne, label="Moyenne des valeurs des pixels")  # Légende pour la moyenne
Map_stats.add_colorbar(vis_params=vis_params_ecart_type, label="Ecart Type des valeurs des pixels")  # Légende pour l'écart type

# Afficher la carte
Map_stats


Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [13]:
# 2. Représenter la situation de chaque pixel en 2020, 2021 et 2022
# ------------------------


# Fonction pour représenter la situation de chaque pixel pour une année donnée
def representer_situation(annee):
    # Importer l'image raster pour l'année spécifiée
    image = importation_rasters(annee)
    
    # Calculer l'indicateur de situation par pixel
    # Catégories : 
    # - Aucune situation grave (valeur < moyenne + écart type)
    # - Situation modérée (moyenne + écart type <= valeur < moyenne + 2 * écart type)
    # - Crise (valeur >= moyenne + 2 * écart type)
    situation = image.expression(
        "(b('b1') < moyenne + ecart_type) ? 1 : (b('b1') < moyenne + 2 * ecart_type) ? 2 : 3",
        {
            'b1': image,
            'moyenne': moyenne_raster,
            'ecart_type': ecart_type_raster
        }
    )
    
    # Clipper l'image à la frontière géographique de la région d'intérêt
    return situation.clip(ee.FeatureCollection("projects/ee-senemalick123450/assets/geoBoundaries-NER-ADM0"))

# Créer une carte pour représenter les situations en 2020, 2021 et 2022
Map_situations = geemap.Map(center=[12, -1], zoom=6)

# Récupérer les images pour les années 2020, 2021 et 2022
situation_2020 = representer_situation(2020)
situation_2021 = representer_situation(2021)
situation_2022 = representer_situation(2022)

# Ajouter les images à la carte avec des paramètres de visualisation
vis_params_situation = {
    'min': 1,
    'max': 3,
    'palette': ['green', 'yellow', 'red']  # 1: aucune situation grave, 2: modérée, 3: crise
}

Map_situations.addLayer(situation_2020, vis_params_situation, 'Situation 2020')
Map_situations.addLayer(situation_2021, vis_params_situation, 'Situation 2021')
Map_situations.addLayer(situation_2022, vis_params_situation, 'Situation 2022')

# Créer un widget HTML pour la légende de la carte
legend_html = """
    <div style='padding: 10px; background-color: white; border-radius: 5px;'>
        <h4>Indicateur de Situation (2020-2022)</h4>
        <p style='margin: 0'><span style='color: green;'>⬤</span> Aucune situation grave</p>
        <p style='margin: 0'><span style='color: yellow;'>⬤</span> Situation modérée</p>
        <p style='margin: 0'><span style='color: red;'>⬤</span> Crise</p>
    </div>
"""
# Créer le widget HTML pour la légende
legend_widget = HTML(value=legend_html)

# Ajouter le widget de légende à la carte
legend_control = WidgetControl(widget=legend_widget, position="topright")
Map_situations.add_control(legend_control)

# Afficher la carte
Map_situations


Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [14]:
# Importer le raster pour l'année 2022
raster_2022 = importation_rasters(2022)

# Calculer l'indicateur de situation par pixel pour 2022
situation_2022 = representer_situation(2022)

# Masquer les autres zones en laissant seulement les taux de malaria pour chaque situation
taux_aucune_situation = raster_2022.updateMask(situation_2022.eq(1))  # Aucune situation grave
taux_situation_moderee = raster_2022.updateMask(situation_2022.eq(2))  # Situation modérée
taux_situation_crise = raster_2022.updateMask(situation_2022.eq(3))    # Crise

# Calculer la population d'enfants pour chaque situation en multipliant les taux par la population d'enfants
# population_enfants_2_10 représente la population totale d'enfants dans la plage d'âge souhaitée
population_enfants_aucune_situation = population_enfants_2_10.multiply(taux_aucune_situation)
population_enfants_situation_moderee = population_enfants_2_10.multiply(taux_situation_moderee)
population_enfants_situation_crise = population_enfants_2_10.multiply(taux_situation_crise)

# Paramètres de visualisation pour les cartes
vis_params_population = {
    'min': 0,
    'max': 50,  # Ajuster selon les valeurs maximales de population dans les données
    'palette': ['white', 'blue']  # Palette pour visualiser la population d'enfants
}

# Créer une carte pour visualiser les populations d'enfants dans chaque situation
Map_population = geemap.Map(center=[12, -1], zoom=6)

# Ajouter les couches pour chaque situation
Map_population.addLayer(population_enfants_aucune_situation, vis_params_population, 'Aucune Situation Grave')
Map_population.addLayer(population_enfants_situation_moderee, vis_params_population, 'Situation Modérée')
Map_population.addLayer(population_enfants_situation_crise, vis_params_population, 'Crise')

# Afficher la carte
Map_population


Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [16]:


# Initialiser Google Earth Engine
ee.Initialize()

# Fonction pour calculer la population d'enfants dans chaque situation pour un niveau administratif
def calculer_population_enfants_par_zone(admin_feature, population_enfants_situation):
    pop_totale = population_enfants_situation.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=admin_feature.geometry(),
        scale=population_enfants_situation.projection().nominalScale(),
        maxPixels=1e13
    ).get('b1')  # 'b1' est la bande par défaut pour les données de population

    # Créer un Feature avec l'ID de la zone et le nombre d'enfants atteints
    return ee.Feature(None, {
        'ID': admin_feature.get('shapeID'),  # 'shapeID' représente l'identifiant de la zone, à adapter selon le cas
        'Enfants_Atteints': pop_totale
    })

# Calculer le nombre d'enfants atteints pour chaque situation et chaque niveau administratif
# Aucune situation grave
population_enfants_aucune_pays = pays.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_aucune_situation))
population_enfants_aucune_regions = regions.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_aucune_situation))
population_enfants_aucune_departements = departements.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_aucune_situation))
population_enfants_aucune_communes = communes.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_aucune_situation))

# Situation modérée
population_enfants_moderee_pays = pays.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_moderee))
population_enfants_moderee_regions = regions.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_moderee))
population_enfants_moderee_departements = departements.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_moderee))
population_enfants_moderee_communes = communes.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_moderee))

# Crise
population_enfants_crise_pays = pays.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_crise))
population_enfants_crise_regions = regions.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_crise))
population_enfants_crise_departements = departements.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_crise))
population_enfants_crise_communes = communes.map(lambda feature: calculer_population_enfants_par_zone(feature, population_enfants_situation_crise))

# Fonction pour convertir une FeatureCollection en dictionnaire pour extraire les ID et Enfants_Atteints
def features_to_dict(feature_collection):
    ids = feature_collection.aggregate_array('ID').getInfo()
    enfants_atteints = feature_collection.aggregate_array('Enfants_Atteints').getInfo()
    return [{'ID': id, 'Enfants_Atteints': enfants} for id, enfants in zip(ids, enfants_atteints)]

# Extraire et exporter les données pour chaque niveau administratif et chaque situation

# Aucune situation grave
data_aucune_pays = features_to_dict(population_enfants_aucune_pays)
data_aucune_regions = features_to_dict(population_enfants_aucune_regions)
data_aucune_departements = features_to_dict(population_enfants_aucune_departements)
data_aucune_communes = features_to_dict(population_enfants_aucune_communes)

# Situation modérée
data_moderee_pays = features_to_dict(population_enfants_moderee_pays)
data_moderee_regions = features_to_dict(population_enfants_moderee_regions)
data_moderee_departements = features_to_dict(population_enfants_moderee_departements)
data_moderee_communes = features_to_dict(population_enfants_moderee_communes)

# Crise
data_crise_pays = features_to_dict(population_enfants_crise_pays)
data_crise_regions = features_to_dict(population_enfants_crise_regions)
data_crise_departements = features_to_dict(population_enfants_crise_departements)
data_crise_communes = features_to_dict(population_enfants_crise_communes)

# Convertir les données en DataFrames et exporter chaque DataFrame en CSV
# Aucune situation grave
pd.DataFrame(data_aucune_pays).to_csv('population_enfants_aucune_situation_pays.csv', index=False)
pd.DataFrame(data_aucune_regions).to_csv('population_enfants_aucune_situation_regions.csv', index=False)
pd.DataFrame(data_aucune_departements).to_csv('population_enfants_aucune_situation_departements.csv', index=False)
pd.DataFrame(data_aucune_communes).to_csv('population_enfants_aucune_situation_communes.csv', index=False)

# Situation modérée
pd.DataFrame(data_moderee_pays).to_csv('population_enfants_moderee_pays.csv', index=False)
pd.DataFrame(data_moderee_regions).to_csv('population_enfants_moderee_regions.csv', index=False)
pd.DataFrame(data_moderee_departements).to_csv('population_enfants_moderee_departements.csv', index=False)
pd.DataFrame(data_moderee_communes).to_csv('population_enfants_moderee_communes.csv', index=False)

# Crise
pd.DataFrame(data_crise_pays).to_csv('population_enfants_crise_pays.csv', index=False)
pd.DataFrame(data_crise_regions).to_csv('population_enfants_crise_regions.csv', index=False)
pd.DataFrame(data_crise_departements).to_csv('population_enfants_crise_departements.csv', index=False)
pd.DataFrame(data_crise_communes).to_csv('population_enfants_crise_communes.csv', index=False)

print("Les fichiers CSV pour chaque situation et niveau administratif ont été exportés avec succès.")


Les fichiers CSV pour chaque situation et niveau administratif ont été exportés avec succès.


In [17]:

# Fonction pour calculer la population totale d'enfants par zone administrative
def calculer_population_totale_par_zone(admin_feature, population_enfants):
    pop_totale = population_enfants.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=admin_feature.geometry(),
        scale=population_enfants.projection().nominalScale(),
        maxPixels=1e13
    ).get('b1')  # 'b1' est la bande par défaut pour les données de population

    # Créer un Feature avec l'ID de la zone et la population totale d'enfants
    return ee.Feature(None, {
        'ID': admin_feature.get('shapeID'),  # 'shapeID' représente l'identifiant de la zone
        'Population_Enfants': pop_totale
    })

# Calculer la population totale d'enfants pour chaque niveau administratif
population_totale_pays = pays.map(lambda feature: calculer_population_totale_par_zone(feature, population_enfants_2_10))
population_totale_regions = regions.map(lambda feature: calculer_population_totale_par_zone(feature, population_enfants_2_10))
population_totale_departements = departements.map(lambda feature: calculer_population_totale_par_zone(feature, population_enfants_2_10))
population_totale_communes = communes.map(lambda feature: calculer_population_totale_par_zone(feature, population_enfants_2_10))

# Fonction pour calculer le taux en fonction des enfants atteints et de la population totale
def calculer_taux_par_zone(enfants_atteints_feature, population_totale_feature):
    id_zone = enfants_atteints_feature.get('ID')
    enfants_atteints = ee.Number(enfants_atteints_feature.get('Enfants_Atteints'))
    population_totale = ee.Number(population_totale_feature.get('Population_Enfants'))
    
    taux = enfants_atteints.divide(population_totale).multiply(100)
    return ee.Feature(None, {
        'ID': id_zone,
        'Taux_Enfants_Atteints': taux
    })

# Appliquer la fonction de taux pour chaque situation et niveau administratif

# Exemple pour "Aucune situation grave" pour les niveaux administratifs
taux_aucune_pays = population_enfants_aucune_pays.map(lambda feature: calculer_taux_par_zone(feature, population_totale_pays.filter(ee.Filter.eq('ID', feature.get('ID'))).first()))
taux_aucune_regions = population_enfants_aucune_regions.map(lambda feature: calculer_taux_par_zone(feature, population_totale_regions.filter(ee.Filter.eq('ID', feature.get('ID'))).first()))
taux_aucune_departements = population_enfants_aucune_departements.map(lambda feature: calculer_taux_par_zone(feature, population_totale_departements.filter(ee.Filter.eq('ID', feature.get('ID'))).first()))
taux_aucune_communes = population_enfants_aucune_communes.map(lambda feature: calculer_taux_par_zone(feature, population_totale_communes.filter(ee.Filter.eq('ID', feature.get('ID'))).first()))

# Convertir les résultats en DataFrames et exporter au format CSV
def taux_to_dict(feature_collection):
    ids = feature_collection.aggregate_array('ID').getInfo()
    taux = feature_collection.aggregate_array('Taux_Enfants_Atteints').getInfo()
    return [{'ID': id, 'Taux_Enfants_Atteints': t} for id, t in zip(ids, taux)]

# Extraire les données pour chaque niveau administratif et exporter les résultats en CSV
data_taux_aucune_pays = taux_to_dict(taux_aucune_pays)
data_taux_aucune_regions = taux_to_dict(taux_aucune_regions)
data_taux_aucune_departements = taux_to_dict(taux_aucune_departements)
data_taux_aucune_communes = taux_to_dict(taux_aucune_communes)

# Exporter chaque DataFrame en CSV
pd.DataFrame(data_taux_aucune_pays).to_csv('taux_enfants_aucune_situation_pays.csv', index=False)
pd.DataFrame(data_taux_aucune_regions).to_csv('taux_enfants_aucune_situation_regions.csv', index=False)
pd.DataFrame(data_taux_aucune_departements).to_csv('taux_enfants_aucune_situation_departements.csv', index=False)
pd.DataFrame(data_taux_aucune_communes).to_csv('taux_enfants_aucune_situation_communes.csv', index=False)

print("Les taux d'enfants atteints de malaria pour 'aucune situation grave' ont été exportés avec succès.")


Les taux d'enfants atteints de malaria pour 'aucune situation grave' ont été exportés avec succès.
